In [2]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from scipy.stats import chi2
import os,sys
import seaborn as sns
sns.set_style('whitegrid')

# import helper script file
## change working directory
os.chdir("C:/Users/rokka/GH-repos/GitHubPages/Code-Reference-Notebook/CU-Boulder/AstroPhys/HW")

## import my own code
import hw_helper_func2 as hf   # this is my own code I made (for probability/distribution fucntions, etc.)

# 3. Find the Signal
There have been many cases in which one must identify a signal buried in noise. It is easier than one thinks! LIGO, for example, offers an example in which distinct types of signals are expected. The data plotted below contains a signal embedded in random noise (fabricated). The data are ina  file "`HW5_Data.txt`" (10000 values) and the signal is in the file "`HW5_Signal.txt`" (400 values).

![](Q3_figures.PNG)

## Part (a)
Read in and verify signals. The cadence is 0.1 sec; generate time array for plotting. Let's start with the easiest method: cross-correlate the expected signal (400 points) with the data (1000 points). You will necessarily need to restrict your correlation to start at 20 sec and end at 980 sec to avoid edge effects. Plot the cross correlation as a function of time. Do you see a peak?

In [6]:
# read in data
data = pd.read_fwf("hw5/HW5_Data.txt", sep=" ",header=None,)
print("data shape:", data.shape)
data.head()

data shape: 2000


,0,1,2,3,4
0,-1.448967,-0.056392,0.102363,0.030122,-0.316273
1,-0.627312,1.505240,1.810492,0.413404,-0.794644
2,0.881385,2.349612,-0.447943,-1.035394,0.926828
3,-0.076224,-1.710956,-0.721547,1.373189,0.363653
4,-1.207440,0.322803,-0.805005,1.421490,0.639479


In [8]:
# Flatten 5d data to 1d
# TODO: could change this to use NumPy from the very beginning
data_1d = data.values.flatten()
print(len(data_1d))
data_1d[:5]

10000


array([-1.4489671 , -0.05639172,  0.10236344,  0.03012188, -0.31627342])

In [7]:
# read in signal data
signal = pd.read_fwf("hw5/HW5_Signal.txt",sep=" ",header=None)
print("signal data shape:", signal.shape)
signal.head()

signal data shape: (80, 5)


,0,1,2,3,4
0,-0.000006,-0.000013,-0.000020,-0.000027,-0.000033
1,-0.000037,-0.000040,-0.000039,-0.000036,-0.000029
2,-0.000018,-0.000004,0.000014,0.000034,0.000056
3,0.000079,0.000101,0.000121,0.000137,0.000148
4,0.000153,0.000148,0.000134,0.000110,0.000074


In [ ]:
# TODO: cross correlate (i.e. the rest of Q3 part a)

## Part (b)
